In [1]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_parameters
from cryptography.hazmat.primitives.serialization import load_pem_public_key

FFDHE2048_PEM = """-----BEGIN DH PARAMETERS-----
MIIBCAKCAQEA//////////+t+FRYortKmq/cViAnPTzx2LnFg84tNpWp4TZBFGQz
+8yTnc4kmz75fS/jY2MMddj2gbICrsRhetPfHtXV/WVhJDP1H18GbtCFY2VVPe0a
87VXE15/V8k1mE8McODmi3fipona8+/och3xWKE2rec1MKzKT0g6eXq8CrGCsyT7
YdEIqUuyyOP7uWrat2DX9GgdT0Kj3jlN9K5W7edjcrsZCwenyO4KbXCeAvzhzffi
7MA0BM0oNC9hkXL+nOmFg/+OTxIy7vKBg8P+OxtMb61zO7X8vC7CIAXFjvGDfRaD
ssbzSibBsu/6iGtCOGEoXJf//////////wIBAg==
-----END DH PARAMETERS-----"""

parameters = load_pem_parameters(FFDHE2048_PEM.encode("utf-8"), backend=default_backend())

# generates a private-public key pair
private_key_a = parameters.generate_private_key()
public_key_a = private_key_a.public_key()

# Serialize public keys for transmission
pem_public_key_a = public_key_a.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
with open("public_key_a.pem", "wb") as pem_file:
    pem_file.write(pem_public_key_a)

In [2]:
def derive_key(public_key_b_file: str):
    try: 
        with open(public_key_b_file, "rb") as f:
            received_public_key_b = load_pem_public_key(f.read())
            
        shared_secret = private_key_a.exchange(received_public_key_b)
        derived_key = HKDF(
                        algorithm=SHA256(),
                        length=32,
                        salt=None,
                        info=b"handshake data",
                        backend=default_backend()
                        ).derive(shared_secret)
        return derived_key
    except:
        print("File read error")

In [3]:
derived_key = derive_key("public_key_a.pem")

In [4]:
def encrypt_message(m: str):
    nonce = os.urandom(12)
    encryptor = Cipher(
                    algorithms.AES(derived_key),
                    modes.GCM(nonce),
                    backend=default_backend()
                ).encryptor()
    ciphertext = encryptor.update(m.encode("utf-8")) + encryptor.finalize()
    auth_tag = encryptor.tag
    
    print(f"Ciphertext: {ciphertext.hex()}")
    print(f"Auth Tag: {auth_tag.hex()}")
    print(f"nonce: {nonce.hex()}")

In [5]:
def decrypt_message(nonce: str, ciphertext: str, tag: str):
    nonce = bytes.fromhex(nonce)
    ciphertext = bytes.fromhex(ciphertext)
    tag = bytes.fromhex(tag)
    decryptor = Cipher(
        algorithms.AES(derived_key),
        modes.GCM(nonce, tag),
        backend=default_backend()
    ).decryptor()

    try:
        plaintext = decryptor.update(ciphertext) + decryptor.finalize()
        print(f"Decrypted Plaintext: {plaintext.decode('utf-8')}")
    except Exception as e:
        print(f"Decryption failed: {str(e)}")

In [6]:
decrypt_message(nonce = "9c89274dd469de6c8b72daff", ciphertext="42a7ab9e442218e19e601da2d1c0f4e751", tag="003d47a80c7e6b6cd96df6d8d06a79e0")

Decryption failed: 


In [1]:
from generator import Generator
g1 = Generator()
g2 = Generator()

In [2]:
g1.DH_keygen()
g2.DH_keygen()
g1.DH_key_exchange(g2.get_public_key())
g2.DH_key_exchange(g1.get_public_key())
g1.decrypt_message(g2.encrypt_message("Hi = 你好"))

'Hi = 你好'